In [ ]:
import numpy as np
import pandas as pd
import scipy.integrate
import scipy.optimize
import scipy.stats
import matplotlib.pyplot as plt

legfont=16
plt.rc("xtick", labelsize=legfont)
plt.rc("ytick", labelsize=legfont)

FMGEV=5.068

def WoodsSaxon(r, R, a):
    ''' r distance from center
        R: nuclear radius
        '''
    return 1/(1 + np.exp((r-R)/a) )

def TA(b, R, a):
    f = lambda z: WoodsSaxon(np.sqrt(z**2+b**2), R, a)
    
    integrated = scipy.integrate.quad(f, -np.inf, np.inf)[0]
    
    norm = 3.0/(4.0 * np.pi * R**3) * 1.0/(1.0 + ( np.pi * a/R)**2)
    
    return norm*integrated

scipy.integrate.quad(lambda b: 2.0*np.pi*b*TA(b, 6*FMGEV, 0.5*FMGEV), 0, np.inf)

TA(1,6*FMGEV, 0.5*FMGEV)

In [ ]:
def ComputeAverage(dirname, maxconf=100):
    data=[]
    for i in range(1,maxconf):
        d = np.loadtxt(dirname + "Run" + str(i) + ".txt")
        
        # note the below function will not work for dataframe as the two lines on top of every Run${i}.txt file\
        # makes it impossible to slice. Use d.shape to see that the # of columns is 1, and hence no slicing in column.
        if np.sum(d[:,2])<1e-10:
            print("Only zeros in file " + i)
            continue
        
        #print (d.shape)
        data.append(d)
        
    err = scipy.stats.sem(data, axis=0)
    
    data = np.mean(data, axis=0)
    
    data = pd.DataFrame(data, columns=["b","N","averageN"])
    
    # noramlize
    interp = scipy.interpolate.interp1d(data["b"],data["averageN"], bounds_error=False, fill_value=0)
    norm = scipy.integrate.quad(lambda b: 2.0*np.pi*b*interp(b), 0, np.inf)[0]
    data["averageN"]/=norm
    data["err"]=err[:,2]/norm
    
    return data


for Y in [4.8]:
    for r in [0.1]:
        data=ComputeAverage(f'Fluctuatingx0_roots200/Y{Y}r{r}/', maxconf=150)
    
        bvals=np.linspace(-50,50)
        popt, pcov = scipy.optimize.curve_fit( np.vectorize(TA), data["b"], data["averageN"], p0=[7*FMGEV,0.5] )

        plt.plot(data["b"]/FMGEV, data["averageN"], label="IPGlasma Pb", color="black")
        plt.fill_between(data["b"]/FMGEV, data["averageN"]-data["err"], data["averageN"]+data["err"],color="black",alpha=0.2)
        plt.plot(bvals/FMGEV, np.vectorize(TA)(bvals, *popt), linestyle="dashed", 
                 label="WS fit, $R=" + str(round(popt[0]/FMGEV,3)) + r"\,\mathrm{fm}, a="+str(round(popt[1]/FMGEV,3)) + r"\,\mathrm{fm}$")
        plt.plot(bvals/FMGEV, np.vectorize(TA)(bvals,7*FMGEV, 0.546*FMGEV), linestyle="dashed", 
                 label="WS fit--our guess R = 7 and a = 0.546" )
        
        plt.xlabel("b [fm]", fontsize=14)
        plt.ylabel(r"$T_A(b),$" + f'r={r}' + r"$\,\mathrm{fm}$", fontsize=14)
        leg = plt.legend(loc="lower center")
        plt.tight_layout()
        plt.savefig(f'Fluctuatingx0_roots200/PLOTS/Y{Y}r{r}.pdf')
        plt.clf()
    


In [6]:
import numpy as np
b=np.linspace(-45.612,45.612,91)


array([-45.612 , -44.5984, -43.5848, -42.5712, -41.5576, -40.544 ,
       -39.5304, -38.5168, -37.5032, -36.4896, -35.476 , -34.4624,
       -33.4488, -32.4352, -31.4216, -30.408 , -29.3944, -28.3808,
       -27.3672, -26.3536, -25.34  , -24.3264, -23.3128, -22.2992,
       -21.2856, -20.272 , -19.2584, -18.2448, -17.2312, -16.2176,
       -15.204 , -14.1904, -13.1768, -12.1632, -11.1496, -10.136 ,
        -9.1224,  -8.1088,  -7.0952,  -6.0816,  -5.068 ,  -4.0544,
        -3.0408,  -2.0272,  -1.0136,   0.    ,   1.0136,   2.0272,
         3.0408,   4.0544,   5.068 ,   6.0816,   7.0952,   8.1088,
         9.1224,  10.136 ,  11.1496,  12.1632,  13.1768,  14.1904,
        15.204 ,  16.2176,  17.2312,  18.2448,  19.2584,  20.272 ,
        21.2856,  22.2992,  23.3128,  24.3264,  25.34  ,  26.3536,
        27.3672,  28.3808,  29.3944,  30.408 ,  31.4216,  32.4352,
        33.4488,  34.4624,  35.476 ,  36.4896,  37.5032,  38.5168,
        39.5304,  40.544 ,  41.5576,  42.5712,  43.5848,  44.5

### """Checking angle dependence for Pb nucleus"""

In [ ]:
def ComputeAverage(dirname, maxconf=100):
    data=[]
    for i in range(1,maxconf):
        d = np.loadtxt(dirname + "Run" + str(i) + ".txt")
        
        if np.sum(d[:,2])<1e-10:
            print("Only zeros in file " + i)
            continue
        
        #print (d.shape)
        data.append(d)
        
    err = scipy.stats.sem(data, axis=0)
    
    data = np.mean(data, axis=0)
    
    data = pd.DataFrame(data, columns=["b","N","averageN"])
    
    # noramlize
    interp = scipy.interpolate.interp1d(data["b"],data["averageN"], bounds_error=False, fill_value=0)
    norm = scipy.integrate.quad(lambda b: 2.0*np.pi*b*interp(b), 0, np.inf)[0]
    data["averageN"]/=norm
    data["err"]=err[:,2]/norm
    
    return data

r=0.3

data=ComputeAverage(f'Fluctuatingx0_roots200/CHECK_ANGLE_DEP/r0.3Theta0/', maxconf=20)

bvals=np.linspace(-50,50)

popt, pcov = scipy.optimize.curve_fit( np.vectorize(TA), data["b"], data["averageN"], p0=[1*FMGEV,1.0] )

plt.plot(data["b"]/FMGEV, data["averageN"], label="IPGlasma Pb", color="black")
plt.fill_between(data["b"]/FMGEV, data["averageN"]-data["err"], data["averageN"]+data["err"],color="black",alpha=0.2)
plt.plot(bvals/FMGEV, np.vectorize(TA)(bvals, *popt), linestyle="dashed", 
         label="WS fit, $R=" + str(round(popt[0]/FMGEV,3)) + r"\,\mathrm{fm}, a="+str(round(popt[1]/FMGEV,3)) + r"\,\mathrm{fm}$")
plt.xlabel("b [fm]", fontsize=14)
plt.ylabel(r"$T_A(b),$" + f'r={r}' + r"$\,\mathrm{fm}$", fontsize=14)
leg = plt.legend(loc="lower center")
plt.tight_layout()

plt.savefig(f'Fluctuatingx0_roots200/CHECK_ANGLE_DEP/PLOTS/r0.3Theta0.pdf')
plt.clf()
    


## Violation of WOOD SAXON

In [ ]:
import numpy as np
import pandas as pd
import scipy.integrate
import scipy.optimize
import scipy.stats
import matplotlib.pyplot as plt

legfont=16
plt.rc("xtick", labelsize=legfont)
plt.rc("ytick", labelsize=legfont)

FMGEV=5.068

def WoodsSaxon(r, R, a):
    ''' r distance from center
        R: nuclear radius
        '''
    
    return 1/(1 + np.exp((r-R)/a) )

def TA(b, R, a):
    f = lambda z: WoodsSaxon(np.sqrt(z**2+b**2), R, a)
    
    integrated = scipy.integrate.quad(f, -np.inf, np.inf)[0]
    
    norm = 3.0/(4.0 * np.pi * R**3) * 1.0/(1.0 + ( np.pi * a/R)**2)
    
    return norm*integrated

scipy.integrate.quad(lambda b: 2.0*np.pi*b*TA(b, 6*FMGEV, 0.5*FMGEV), 0, np.inf)



In [ ]:
def ComputeAverage(dirname, maxconf=100):
    data=[]
    for i in range(1,maxconf):
        d = np.loadtxt(dirname + "Run" + str(i) + ".txt")
        
        if np.sum(d[:,2])<1e-10:
            print("Only zeros in file " + i)
            continue
        
        data.append(d)
        
    err = scipy.stats.sem(data, axis=0)
    
    data = np.mean(data, axis=0)
    
    data = pd.DataFrame(data, columns=["b","N","averageN"])
    
    """ NORMALIZING IT HERE FOR GETTING RID OF V0 IN THE NUMERATOR"""
    #### noramlize
    interp = scipy.interpolate.interp1d(data["b"],data["averageN"], bounds_error=False, fill_value=0)
    norm = scipy.integrate.quad(lambda b: 2.0*np.pi*b*interp(b), 0, np.inf)[0]
    data["averageN"]/=norm
    data["err"]=err[:,2]/norm
    
    return data


"""SAVING THE NORMALIZED BESTFIT DATA FOR Y = 0 AND r = 0.1 fm"""
Bestfit=ComputeAverage(f'Fluctuatingx0_roots200/Y0r0.1/', maxconf=150)

popt, pcov = scipy.optimize.curve_fit(np.vectorize(TA), Bestfit["b"], Bestfit["averageN"], p0=[1*FMGEV,1.0] )
Bestfit["averageN"]=np.vectorize(TA)(Bestfit["b"], *popt)
Bestfit = Bestfit.drop(columns={"N", "err"})

Bestfit.to_csv(f'Fluctuatingx0_roots200/BestfitY0r0.1.txt', sep=' ', index=False)

WS_violate = []
for Y in [0, 2.4, 4.8, 7.2, 9.6]:
    for r in [0.05, 0.1]:
        data=ComputeAverage(f'Fluctuatingx0_roots200/Y{Y}r{r}/', maxconf=150)
        
        """NORMALISING FOR THE WOODSAXON VIOLATION"""
        interp = scipy.interpolate.interp1d(data["b"],abs(data["averageN"]-Bestfit["averageN"]), bounds_error=False, fill_value=0)
        val = scipy.integrate.quad(lambda b: 2.0*np.pi*b*interp(b), 0, np.inf)[0]
        WS_violate.append((Y,r,val))
        


In [ ]:
chkdata=ComputeAverage(f'Fluctuatingx0_roots200/Y9.6r0.1/', maxconf=150)

In [ ]:
chkdata.to_csv('Fluctuatingx0_roots200/Chky9.6r0.1avg.txt', sep=' ', index=False)
Bestfit=pd.read_csv('Fluctuatingx0_roots200/BestfitY0r0.1.txt',  delimiter=' ' )

In [ ]:
"""checking the Violation for Y9.6r0.l case"""
areaa = 0.0
db=1.0136
i=0
for b in blist: #taking pi instead of 2 pi because data would be symmetric and hence in normalizing the WS part, integrated over 0 to infinity, and not on -infinity to infinity
    areaa += np.pi * abs(chkdata.loc[i,"b"]) * db * abs(Bestfit.loc[i,"averageN"] - chkdata.loc[i,"averageN"])
    i+=1
print(areaa)

